# Импорт необходимых модулей

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import math

# На всякий случай...

[ISBN](https://en.wikipedia.org/wiki/International_Standard_Book_Number)

# Чтение данных

In [ ]:
ratings = pd.read_csv('data/BX-Book-Ratings.csv', sep=';', header=0, error_bad_lines=False, encoding='Windows-1251', low_memory=False)

In [ ]:
books = pd.read_csv('data/BX-Books.csv', sep=';', header=0, error_bad_lines=False, encoding='Windows-1251', low_memory=False)

In [ ]:
users = pd.read_csv('data/BX-Users.csv', sep=';', header=0, error_bad_lines=False, encoding='Windows-1251', low_memory=False)

# Знакомство

Ок, загрузили, надо теперь поглядеть, что это вообще за данные.

In [ ]:
ratings.head()

In [ ]:
ratings.info()

In [ ]:
books.head()

Последние три признака не являются информативными, поскольку представляют собой ссылки на картинки обложек книг разных размеров. Их можно удалить:

In [ ]:
books = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher']]

Гдянем теперь, нет ли каких-нибудь повторений среди этих книг:

In [ ]:
books_groupped = books.groupby('Book-Title').agg([np.array, len])

In [ ]:
books_groupped.sort_values(by=[('ISBN', 'len')], ascending=False).head(50)

Очевидно есть повторения.

Заметно, что по некоторым публикациям не указан год издания, также по некоторым книгам имена авторов по-разному указаны.

В остальном это разные издания одних и тех же работ, напечатанных в разные (не все) годы разными издательствами.

Пока не совсем понятно, что делать с этой информацией.

Попробуем глянуть, какие есть оценки для разных изданий какого-нибудь произведения:

In [ ]:
ratings[ratings.ISBN.isin(books_groupped.loc['Jane Eyre', ('ISBN', 'array')])][ratings['Book-Rating'] > 0]['User-ID'].nunique()

Итак, в частности для произведения **"Джейн Эйр"** присутствует довольно большое количество оценок от разных пользователей (для разных ISBN).

Собственно, логично предположить, что, оценивая книгу, читатель оценивает далеко не только само произведение, но еще и обложку, шрифт, цвет, качество бумаги (если это печатное издание).

Существуют, наверное и другие факторы, не имеющие отношения к самому произведению, однако здесь это особо не проверить, поскольку данных на эту тему нет.

In [ ]:
books.info()

In [ ]:
users.head()

In [ ]:
users.info()

Итак, у нас есть 271379 книг и 278858 пользователей, по книгам известны их:
- ISBN;
- название;
- автор;
- год выпуска;
- издательство.

По пользователям (не по всем) известно их:
- местоположение;
- возраст.

Надо глянуть какой процент пользователей указали возраст:

In [ ]:
100 * len(users[~users.Age.isnull()]) / len(users)

Ок, ~60% пользователей не стесняются, чего не скажешь об остальных.

Глянем, как читатели распределены по возрасту:

In [ ]:
plt.figure(figsize=(35, 15))
n, bins, _ = plt.hist(users[~users.Age.isnull()].Age, bins=20)
plt.xticks(bins)
plt.show()

Что же, очевидно, есть некоторая часть пользователей с возрастом, указанным не вполне адекватно:

In [ ]:
users[~users.Age.isnull()].Age.min()

In [ ]:
users[~users.Age.isnull()].Age.max()

Вряд ли кто-то в возрасте 0 или 244 лет что-то покупал из книг.

Заметим, что поле Location состоит из трех пунктов: город, регион, страна.

Надо бы разделить это поле на три, чтобы глянуть, как пользователи по миру раскиданы.

Перед этим надо глянуть, можно ли вообще легко это разделение сделать:

In [ ]:
locations = users.Location.values
locations = np.array([loc.split(', ') for loc in locations])

In [ ]:
incorrect_items_more = [ind for ind, item in enumerate(locations) if len(item) > 3]
incorrect_items_less = [ind for ind, item in enumerate(locations) if len(item) < 3]

In [ ]:
print(100 * (len(incorrect_items_more) + len(incorrect_items_less)) / len(locations))

~2% людей с коряво указанными данными по местоположению.

Надо глянуть, сколько из них оценили какие-нибудь книги:

In [ ]:
users_with_their_ratings_l = pd.merge(users[users.index.isin(incorrect_items_less)], ratings, how='inner', on='User-ID')

In [ ]:
plt.figure(figsize=(16, 16))
users_with_their_ratings_l['Book-Rating'].value_counts().plot.pie(autopct='%.2f')
plt.show()

In [ ]:
users_with_their_ratings_m = pd.merge(users[users.index.isin(incorrect_items_more)], ratings, how='inner', on='User-ID')

In [ ]:
plt.figure(figsize=(8, 8))
users_with_their_ratings_m['Book-Rating'].value_counts().plot.pie(autopct='%.2f')
plt.show()

Здесь графики приводятся в разных масштабах, чтобы не ввести в заблуждение по поводу объема выборок пользователей с коряво указанными локациями:

In [ ]:
print(len(incorrect_items_more), len(incorrect_items_less))

Надо бы еще глянуть, как много ISBN, указанных в таблице BX-Book-Ratings, отсутствуют в таблице BX-Books.

То же самое по поводу пользователей:

In [ ]:
ratings[~ratings.ISBN.isin(books.ISBN)].ISBN.nunique()

Получается, что в списке книг есть такие, о которых, вообще говоря, ничего не известно.

Надо глянуть, по скольким таким книгам есть оценки:

In [ ]:
non_zero_ratings = ratings[ratings['Book-Rating'] > 0]
non_zero_ratings[~non_zero_ratings.ISBN.isin(books.ISBN)].ISBN.nunique()

Что же, у нас есть оценки пользователей по книгам, о которых мы ничего не знаем.

Глянем, как это количество соотносится с общим количеством оцененных пользователями книг:

In [ ]:
100 * non_zero_ratings[~non_zero_ratings.ISBN.isin(books.ISBN)].ISBN.nunique() / non_zero_ratings.ISBN.nunique()

С одной стороны 19% - это довольно много, с другой - это 19% оценок книг, о которых ничего неизвестно.

Это просто какие-то номера, по которым нельзя будет, скажем построить какой-нибудь Item-based алгоритм, после применения Collaborative filtering нельзя будет сказать, каким именно книгам были восстановлены оценки, нельзя будет сделать никаких выводов по этому поводу.

Собственно поэтому логичнее эти оценки пока не рассматривать.

In [ ]:
non_zero_ratings = non_zero_ratings[non_zero_ratings.ISBN.isin(books.ISBN)]

In [ ]:
non_zero_ratings.ISBN.nunique()

In [ ]:
non_zero_ratings[~non_zero_ratings['User-ID'].isin(users['User-ID'])]['User-ID'].nunique()

Итак, нет ни одного ID для которого бы в таблице users не было бы записи.

Хорошо, теперь надо еще ответить на обратные вопросы: есть ли такие пользователи, которые не дали оценки ни одной книге и есть ли такие книги, которые не были оценены ни одним пользователем:

In [ ]:
100 * users[~users['User-ID'].isin(non_zero_ratings['User-ID'])]['User-ID'].nunique() / users['User-ID'].nunique()

In [ ]:
100 * books[~books.ISBN.isin(non_zero_ratings.ISBN)].ISBN.nunique() / books.ISBN.nunique()

Получается, у нас есть ~75.6% пользователей, никак ни одной книги не оценивших и ~44.8% книг, никах никем не оценённых.

# К вопросу о правильном Evaluation

Здесь общая идея следующая:
- сформировать выборку троек (пользователь, книга, оценка), желательно, чтобы эта выборка покрыла как можно больше книг;
- извлечь эти тройки из основного набора данных (оценок) и далее считать, что для выбранных пользователей и книг оценки неизвестны и должны быть восстановлены;
- собственно, применить один из алгоритмов восстановления оценок;
- пользуясь выбраннной метрикой (RMSE или NDCG) оценить качество восстановления пользовательских оценок.

# Сколько читателей поставили оценки и сколько книг оценили

Ок, теперь надо глянуть, какие пользователи сколько раз поставили оценки и какие книги сколько раз были оценены.

Это необходимо для того, чтобы понять как разделять датасет на трейн и тест.

Также нужно понять какую часть пользователей и книг нужно отсеять (пользователи, оценившие слишком мало книг из всего множества, книги, оцененные слишком мало пользователями).

In [ ]:
non_zero_ratings_groupped_by_user = non_zero_ratings.groupby('User-ID').agg(len)

In [ ]:
non_zero_ratings_groupped_by_user.sort_values(by=['ISBN'], ascending=False).head(10)

In [ ]:
len(non_zero_ratings_groupped_by_user[non_zero_ratings_groupped_by_user.ISBN > 10])

In [ ]:
non_zero_ratings_groupped_by_user[non_zero_ratings_groupped_by_user.ISBN > 10].index.values

In [ ]:
non_zero_ratings[non_zero_ratings['User-ID'] == 278633].index.values

In [ ]:
len(non_zero_ratings_groupped_by_user)

In [ ]:
non_zero_ratings_groupped_by_isbn = non_zero_ratings.groupby('ISBN').agg(len)

In [ ]:
non_zero_ratings_groupped_by_isbn.sort_values(by=['User-ID'], ascending=False).head(10)

In [ ]:
len(non_zero_ratings_groupped_by_isbn[non_zero_ratings_groupped_by_isbn['User-ID'] > 10])

In [ ]:
len(non_zero_ratings_groupped_by_isbn)